In [10]:
import pandas as pd

# Load the CSV file
file_path = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Odisha-Risk-Score-Model\data\risk_score.csv"
print("Loading CSV file from:", file_path)
df = pd.read_csv(file_path)
print("CSV file loaded successfully. Shape:", df.shape)

Loading CSV file from: D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Odisha-Risk-Score-Model\data\risk_score.csv
CSV file loaded successfully. Shape: (13816, 83)


In [11]:
# Selecting only numeric columns along with 'object-id', 'timeperiod', and 'financial-year'
numeric_columns = df.select_dtypes(include=["number"]).columns
list(numeric_columns).remove("block-area")
numeric_columns = numeric_columns.drop("block-area")
df_numeric = df[["object-id", "timeperiod", "financial-year"] + list(numeric_columns)]

# Identifying columns that are not included
excluded_columns = [col for col in df.columns if col not in df_numeric.columns]
print("Excluded columns:", excluded_columns)

df_numeric.head()

Excluded columns: ['block-area', 'district', 'dtname', 'block-name']


,object-id,timeperiod,financial-year,sdrf-sanctions-awarded-value,total-tender-awarded-value,ridf-tenders-awarded-value,preparedness-measures-tenders-awarded-value,immediate-measures-tenders-awarded-value,others-tenders-awarded-value,id,...,flood-hazard,government-response,vulnerability,total-tender-awarded-value-fy-cumsum,preparedness-measures-tenders-awarded-value-fy-cumsum,immediate-measures-tenders-awarded-value-fy-cumsum,others-tenders-awarded-value-fy-cumsum,sdrf-sanctions-awarded-value-fy-cumsum,topsis-score,risk-score
0,21-399-03482,2021_04,2021-2022,8931251,0.00,0.0,0.0,0.0,0.0,0,...,4,2,5,0.00,0.0,0.0,0.0,8931251,0.788429,5
1,21-378-03326,2021_04,2021-2022,457603426,0.00,0.0,0.0,0.0,0.0,0,...,4,1,4,0.00,0.0,0.0,0.0,457603426,0.710521,5
2,21-375-03455,2021_04,2021-2022,46110192,7056275.72,0.0,0.0,0.0,0.0,0,...,4,2,5,7056275.72,0.0,0.0,0.0,46110192,0.707290,5
3,21-399-03484,2021_04,2021-2022,8931251,0.00,0.0,0.0,0.0,0.0,0,...,4,2,5,0.00,0.0,0.0,0.0,8931251,0.707290,5
4,21-375-03443,2021_04,2021-2022,46110192,3682534.00,0.0,0.0,0.0,1983261.0,0,...,4,2,5,3682534.00,0.0,0.0,1983261.0,46110192,0.707290,5


In [12]:
df_melted = df_numeric.melt(id_vars=["object-id", "timeperiod", "financial-year"], var_name="factor", value_name="score")
print("Data melted. Shape:", df_melted.shape)

df_melted.head()

Data melted. Shape: (1050016, 5)


,object-id,timeperiod,financial-year,factor,score
0,21-399-03482,2021_04,2021-2022,sdrf-sanctions-awarded-value,8931251.0
1,21-378-03326,2021_04,2021-2022,sdrf-sanctions-awarded-value,457603426.0
2,21-375-03455,2021_04,2021-2022,sdrf-sanctions-awarded-value,46110192.0
3,21-399-03484,2021_04,2021-2022,sdrf-sanctions-awarded-value,8931251.0
4,21-375-03443,2021_04,2021-2022,sdrf-sanctions-awarded-value,46110192.0


In [6]:
df_melted.columns

Index(['object-id', 'timeperiod', 'financial-year', 'factor', 'score'], dtype='object')

In [13]:
df_transposed = df_melted.pivot(index=["factor", "timeperiod", "financial-year"], columns="object-id", values="score").reset_index()
print("Data pivoted. Shape:", df_transposed.shape)
df_transposed.head()
# Save or display the transformed data
output_file = "Transformed_Assam_Data.csv"
df_transposed.to_csv(output_file, index=False)
print("Transformed data saved to:", output_file)

Data pivoted. Shape: (3344, 317)
Transformed data saved to: Transformed_Assam_Data.csv


In [14]:
# see how many duplicates you have:
dups = df_melted.duplicated(
    subset=["factor", "timeperiod", "financial-year", "object-id"],
    keep=False
)
print("Total duplicate rows:", dups.sum())
print(df_melted[dups].sort_values(
    ["factor", "timeperiod", "financial-year", "object-id"]
).head())


Total duplicate rows: 0
Empty DataFrame
Columns: [object-id, timeperiod, financial-year, factor, score]
Index: []


In [7]:
# Verify with the source data. Given factor, district, timeperiod and object-id, the score should match

factor = "sdrf-sanctions-awarded-value"
district = '18-319-00202'
timeperiod = '2024_07'
df_transposed.head()
print("modified")
print(df_transposed[(df_transposed["factor"] == factor) & (df_transposed["timeperiod"] == timeperiod) ][[district, "timeperiod", "financial-year", "factor"]])
print("original")
df[(df["object-id"] == district) & (df["timeperiod"] == timeperiod)][['timeperiod', 'financial-year', factor]]



modified
object-id  18-319-00202 timeperiod financial-year  \
2874                0.0    2024_07      2024-2025   

object-id                        factor  
2874       sdrf-sanctions-awarded-value  
original


,timeperiod,financial-year,sdrf-sanctions-awarded-value
7138,2024_07,2024-2025,0.0


In [7]:
dff = pd.read_csv(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Assam-Risk-Model\RiskScoreModel\data\Transformed_Assam_Data.csv")
conditions = []
operator_map = {
            '==': lambda col, val: col == val,
            '!=': lambda col, val: col != val,
            '>': lambda col, val: col > val,
            '<': lambda col, val: col < val,
            '>=': lambda col, val: col >= val,
            '<=': lambda col, val: col <= val,
            'in': lambda col, val: col.isin(val),
            'not in': lambda col, val: ~col.isin(val)
        }
conditions.append(operator_map["=="](dff["factor"], "sdrf-sanctions-awarded-value"))

dff = dff[pd.concat(conditions, axis=1).all(axis=1)]
dff

,factor,timeperiod,financial-year,18-300,18-300-00101,18-300-00102,18-300-00103,18-300-00104,18-300-00105,18-300-00106,...,18-799,18-799-00124,18-799-00125,18-799-00265,18-816,18-816-00235,18-816-00236,18-816-00258,18-816-00259,18-816-00261
3360,sdrf-sanctions-awarded-value,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3361,sdrf-sanctions-awarded-value,2021_05,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3362,sdrf-sanctions-awarded-value,2021_06,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3363,sdrf-sanctions-awarded-value,2021_07,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3364,sdrf-sanctions-awarded-value,2021_08,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3365,sdrf-sanctions-awarded-value,2021_09,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3366,sdrf-sanctions-awarded-value,2021_10,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3367,sdrf-sanctions-awarded-value,2021_11,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3368,sdrf-sanctions-awarded-value,2021_12,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369,sdrf-sanctions-awarded-value,2022_01,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
